In [28]:
import pandas as pd

In [29]:
"""
Loads in raw score file
Expecting the schema 
Name	Date	GF	GA	Team

Calcuates result, margies and goal difference
Does some prep work for Form and Win

Obviously names need to be consistant
"""

raw_data = pd.read_csv("../data/results_v2.csv")
raw_data["GD"] = raw_data["GF"] - raw_data["GA"]
raw_data.loc[(raw_data['GD'] == 0), 'GamePoints'] = 0.5
raw_data.loc[(raw_data['GD'] < 0), 'GamePoints'] = -1
raw_data.loc[(raw_data['GD'] > 0), 'GamePoints'] = 1
raw_data.loc[(raw_data['GD'] > 6), 'GamePoints'] = 1.5
raw_data.loc[(raw_data['GD'] < -6), 'GamePoints'] = -1.5
raw_data.loc[(raw_data['GamePoints'] >= 1), 'W'] = 1
raw_data.loc[(raw_data['GamePoints'] == 0.5), 'D'] = 1
raw_data.loc[(raw_data['GamePoints'] <= -1), 'L'] = 1
raw_data.loc[(raw_data['GamePoints'] == 1.5), 'MW'] = 1
raw_data.loc[(raw_data['GamePoints'] == -1.5), 'ML'] = 1

raw_data.loc[(raw_data['GamePoints'] >= 1), 'Result'] = '🔼'
raw_data.loc[(raw_data['GamePoints'] == 0.5), 'Result'] = '➖'
raw_data.loc[(raw_data['GamePoints'] <= -1), 'Result'] = '🔽'
raw_data.loc[(raw_data['GamePoints'] == 1.5), 'Result'] = '⏫'
raw_data.loc[(raw_data['GamePoints'] == -1.5), 'Result'] = '⏬'

raw_data = raw_data[raw_data["Name"].str.contains("\*")==False] # Removes Ringers

raw_data = raw_data.fillna(0)
pd.set_option('display.float_format', '{:.1f}'.format)
raw_data = raw_data.astype({'W':'int','D':'int','L':'int','MW':'int','ML':'int'})

raw_data['Date'] = pd.to_datetime(raw_data['Date'],dayfirst = True)
gameweek_int = raw_data.nunique().Date
gameweek = "Gameweek " + str(gameweek_int)

#display(raw_data)

In [30]:
"""
Creates the Form Column
"""
def form_column(raw_data):
    raw_data['game_rank'] = raw_data.sort_values("Date").groupby("Name").Date.rank(ascending=False)
    raw_data_form = raw_data.loc[raw_data['game_rank'] <= 5]
    form = raw_data_form.groupby(['Name'], as_index=False).agg({'Result': ' '.join})
    return form

form = form_column(raw_data)

#display(form)

In [31]:
"""
Creates Matches Played (MP), Partipation (Par%), Pts and Win% (W%)

It also aggreates all the raw data to create an up to date table
"""
def get_totals(raw_data):
    played = raw_data.groupby('Name').count()
    played = pd.DataFrame(played["Date"]).rename(columns={"Date":"MP"})
    data = raw_data.groupby('Name').sum(numeric_only=True).sort_values(by=['GamePoints'],ascending=False).join(played).rename(columns={"GamePoints":"Pts"})

    data["Par%"] = data["MP"].div(gameweek_int)*100
    data["W%"] = data["W"]/data["MP"]*100
    data = data.astype({'Par%':'int','W%':'int'})
    return data

totals = get_totals(raw_data)



#display(data)


In [32]:
"""
Join tables 
Drops unwanted columns 
Puts columns in the correct order
"""
table = totals.merge(form,on='Name').rename(columns={"Result":"Form"})
table.index = table.Name
table = table[['MP', 'W', 'D', 'L','MW', 'ML', 'GF', 'GA','GD','Par%','W%','Pts','Form']]
table.index.names = [gameweek]
table

MP   W  D   L  MW  ML   GF   GA  GD  Par%  W%  Pts       Form
Gameweek 30                                                               
Calz         18  11  1   6   5   1  278  227  51    60  61  7.5  ⏫ ➖ 🔼 🔽 🔼
Schurrle     18  11  0   7   3   2  272  252  20    60  61  4.5  🔼 🔽 🔼 🔼 🔽
Zola         17   9  1   7   3   2  241  246  -5    56  52  3.0  ➖ 🔽 🔽 🔼 🔼
Reaper       24  14  0  10   2   5  332  343 -11    80  58  2.5  🔼 🔽 🔼 🔽 🔼
Virlo        13   7  1   5   2   2  191  186   5    43  53  2.5  ➖ 🔽 🔽 🔼 🔼
Gazprom      20  11  0   9   2   3  279  280  -1    66  55  1.5  🔽 🔼 🔼 🔽 🔽
Peggy         6   3  0   3   1   0   90   79  11    20  50  0.5  🔽 🔼 ⏫ 🔽 🔼
Helmet       22  11  0  11   3   4  299  319 -20    73  50 -0.5  🔼 🔼 🔽 🔼 🔽
Keysey       11   4  1   6   1   1  154  151   3    36  36 -1.5  ➖ 🔼 🔽 🔽 🔽
Snoop        18   8  0  10   3   2  247  236  11    60  44 -1.5  🔽 🔼 🔼 🔼 🔼
Gilet        15   6  0   9   2   3  207  231 -24    50  40 -3.5  🔽 🔽 🔼 🔽 🔼
Golfer        7   2  0   5   0   2   85  104 -19    23  28 -4.0  🔽 🔼 🔽 🔽 🔼
Kim          26  10  1  15   3   3  357  382 -25    86  38 -4.5  🔽 🔽 🔼 🔽 🔽
Mate         26   9  1  16   4   3  366  361   5    86  34 -6.0  🔼 🔽 🔽 🔽 🔽
Noel         18   5  1  12   2   5  223  275 -52    60  27 -8.0  ⏬ ➖ 🔼 🔽 🔽

In [33]:
"""
TO DO

Add Graphics
Tracking player pairings
Tracking week by week table position

"""


'\nTO DO\n\nAdd Graphics\nTracking player pairings\nTracking week by week table position\n\n'

In [34]:
display(raw_data)

Name       Date  GF  GA     Team  GD  GamePoints  W  D  L  MW  ML  \
0        Calz 2023-03-02  16  17    DARKS  -1        -1.0  0  0  1   0   0   
1      Helmet 2023-03-02  16  17    DARKS  -1        -1.0  0  0  1   0   0   
2         Kim 2023-03-02  16  17    DARKS  -1        -1.0  0  0  1   0   0   
3        Mate 2023-03-02  16  17    DARKS  -1        -1.0  0  0  1   0   0   
4       Peggy 2023-03-02  16  17    DARKS  -1        -1.0  0  0  1   0   0   
..        ...        ...  ..  ..      ...  ..         ... .. .. ..  ..  ..   
293  Schurrle 2023-09-14  10  12    DARKS  -2        -1.0  0  0  1   0   0   
295      Zola 2023-09-14  12  10  BLANCOS   2         1.0  1  0  0   0   0   
296    Golfer 2023-09-14  12  10  BLANCOS   2         1.0  1  0  0   0   0   
297     Snoop 2023-09-14  12  10  BLANCOS   2         1.0  1  0  0   0   0   
298    Reaper 2023-09-14  12  10  BLANCOS   2         1.0  1  0  0   0   0   

    Result  game_rank  
0        🔽       18.0  
1        🔽       22.0  
2        🔽       26.0  
3        🔽       26.0  
4        🔽        6.0  
..     ...        ...  
293      🔽        1.0  
295      🔼        1.0  
296      🔼        1.0  
297      🔼        1.0  
298      🔼        1.0  

[259 rows x 14 columns]

In [35]:
def count_same_team(df, name_col, date_col, team_col):
    count = 0
    for idx, row in df.iterrows():
        name = row[name_col]
        date = row[date_col]
        team = row[team_col]
        same_team_df = df[(df[team_col] == team) & (df[date_col] == date)]
        same_team_names = same_team_df[name_col].unique()
        if len(same_team_names) > 1 and name in same_team_names:
            print(same_team_names)
    return count

In [36]:
count_same_team(raw_data, 'Name', 'Date', 'Team')

['Calz' 'Helmet' 'Kim' 'Mate' 'Peggy']
['Calz' 'Helmet' 'Kim' 'Mate' 'Peggy']
['Calz' 'Helmet' 'Kim' 'Mate' 'Peggy']
['Calz' 'Helmet' 'Kim' 'Mate' 'Peggy']
['Calz' 'Helmet' 'Kim' 'Mate' 'Peggy']
['Gilet' 'Reaper' 'Schurrle' 'Keysey' 'Gazprom']
['Gilet' 'Reaper' 'Schurrle' 'Keysey' 'Gazprom']
['Gilet' 'Reaper' 'Schurrle' 'Keysey' 'Gazprom']
['Gilet' 'Reaper' 'Schurrle' 'Keysey' 'Gazprom']
['Gilet' 'Reaper' 'Schurrle' 'Keysey' 'Gazprom']
['Gilet' 'Helmet' 'Mate' 'Noel' 'Reaper']
['Gilet' 'Helmet' 'Mate' 'Noel' 'Reaper']
['Gilet' 'Helmet' 'Mate' 'Noel' 'Reaper']
['Gilet' 'Helmet' 'Mate' 'Noel' 'Reaper']
['Gilet' 'Helmet' 'Mate' 'Noel' 'Reaper']
['Calz' 'Kim' 'Virlo' 'Gazprom' 'Snoop']
['Calz' 'Kim' 'Virlo' 'Gazprom' 'Snoop']
['Calz' 'Kim' 'Virlo' 'Gazprom' 'Snoop']
['Calz' 'Kim' 'Virlo' 'Gazprom' 'Snoop']
['Calz' 'Kim' 'Virlo' 'Gazprom' 'Snoop']
['Gilet' 'Schurrle' 'Virlo' 'Gazprom']
['Gilet' 'Schurrle' 'Virlo' 'Gazprom']
['Gilet' 'Schurrle' 'Virlo' 'Gazprom']
['Gilet' 'Schurrle' 'Virlo' 

['Gazprom' 'Gilet' 'Keysey' 'Golfer']
['Gazprom' 'Gilet' 'Keysey' 'Golfer']
['Gazprom' 'Gilet' 'Keysey' 'Golfer']
['Zola' 'Gilet' 'Snoop']
['Zola' 'Gilet' 'Snoop']
['Zola' 'Gilet' 'Snoop']
['Kim' 'Mate' 'Reaper' 'Golfer']
['Kim' 'Mate' 'Reaper' 'Golfer']
['Kim' 'Mate' 'Reaper' 'Golfer']
['Kim' 'Mate' 'Reaper' 'Golfer']
['Mate' 'Kim' 'Helmet' 'Schurrle']
['Mate' 'Kim' 'Helmet' 'Schurrle']
['Mate' 'Kim' 'Helmet' 'Schurrle']
['Mate' 'Kim' 'Helmet' 'Schurrle']
['Zola' 'Golfer' 'Snoop' 'Reaper']
['Zola' 'Golfer' 'Snoop' 'Reaper']
['Zola' 'Golfer' 'Snoop' 'Reaper']
['Zola' 'Golfer' 'Snoop' 'Reaper']


0